# Wikipediaから一覧表を取得

## Setup

In [1]:
__author__ = "Admin GGCS"
__copyright__ = "Copyright 2019"
__license__ = "MIT"
__version__ = "2.0.0"
__maintainer__ = "Admin GGCS"
__website__ = "ggcs.io"

import requests
import pandas as pd
import os

## Main Part

### 対象URL入力、requests.models.Responseに変換

In [2]:
#pd.read_html()は多くの場合は直接ページ（myUrl）を読み込めるけれど、
#相手のencodingの関係でエラーになる場合もあるので念のためrequests.models.Responseに変換しておく。
try:
    myUrl = input('URL? (ex. https://www.example.com) >> ')
    response = requests.get(myUrl)
except Exception as exc_msg:
        print("Error!{}".format(exc_msg))

#入力例: https://ja.wikipedia.org/wiki/生物の分類

URL? (ex. https://www.example.com) >> https://ja.wikipedia.org/wiki/生物の分類


### 🐼 I love Pandas!

In [3]:
#カッコ内はresponse.textでも良いが、response.contentの方が良さげ。
#っていうか、pd.read_html()なんて凄いのがあるのを一昨日の夜に知った。早く教えてよ😭
myList = pd.read_html(response.content)

In [4]:
#myListのtypeはlistなのに、切り出すとData Frame になる不思議。
#こんな感じ。
print('myList   : ', type(myList), '\nmyList[0]: ', type(myList[0]))

myList   :  <class 'list'> 
myList[0]:  <class 'pandas.core.frame.DataFrame'>


### これで良いか、printして様子を見てみる。

In [5]:
#下のセルに表示されているのは、'https://ja.wikipedia.org/wiki/生物の分類'をスクレイピングした時の例
for i,j in enumerate(myList):
    print('[Table ' + str(i).zfill(3) + ']\n', j)

[Table 000]
     0                                                  1
0 NaN  ウィキペディアにおける生物の分類の扱いについては、Wikipedia:ウィキプロジェクト 生...
[Table 001]
                0               1                 2              3  \
0  リンネ（1735年）2界説  ヘッケル（1894年）3界説  ホイタッカー（1969年）5界説  ウーズ（1977年）6界説   
1            NaN           原生生物界              モネラ界          真正細菌界   
2            NaN           原生生物界              モネラ界           古細菌界   
3            NaN           原生生物界             原生生物界          原生生物界   
4            植物界             植物界                菌界             菌界   
5            植物界             植物界               植物界            植物界   
6            動物界             動物界               動物界            動物界   

                     4             5  
0  ウーズ（1990年）3ドメイン説[6]        具体例[7]  
1                   細菌  大腸菌、放線菌、藍色細菌  
2            古細菌（アーキア）  メタン生成菌、好熱好酸菌  
3                 真核生物  藻類、原生動物、変形菌類  
4                 真核生物   キノコ、カビ、地衣植物  
5                 真核生物  コケ類、シダ類、種子植物  
6                 真核生物    無脊椎動物、脊椎

### これで良いのだ。CSVファイルに書き出す。おしまい。

In [7]:
try:
    goornogo =  input(str(len(myList))+'　csv files will be generated. OK? ([y]/n) >>  ').lower()
    if goornogo != 'n':
        if os.path.isfile('output_001.csv'):
            print('Oops! output_001.csv already exixts!\n*** ABORTED ***')
            os.system('open .')
        else:
            for i,j in enumerate(myList):
                j.to_csv('output_'+str(i).zfill(3)+'.csv')
            print('Done!')
            os.system('open .')
    else: print('Bye!')
except Exception as exc_msg:
        print("Error!{}".format(exc_msg))

7　csv files will be generated. OK? ([y]/n) >>  y
Done!


### え？Excelがいいの？

In [8]:
try:
    goornogo =  input(str(len(myList))+'　xlsx files will be generated. OK? ([y]/n) >>  ').lower()
    if goornogo != 'n':
        if os.path.isfile('output_001.xlsx'):
            print('Oops! output_001.xlsx already exixts!\n*** ABORTED ***')
            os.system('open .')
        else:
            for i,j in enumerate(myList):
                j.to_excel('output_'+str(i).zfill(3)+'.xlsx')
            print('Done!')
            os.system('open .')
    else: print('Bye!')
except Exception as exc_msg:
        print("Error!{}".format(exc_msg))

7　xlsx files will be generated. OK? ([y]/n) >>  y
Done!
